In [1]:
import json
from datasets import Dataset

# Load your JSON file
with open('./train-v1.1.json') as f:
    data = json.load(f)

# Convert your data to a list of dictionaries
examples = []
for item in data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answer = qa['answers'][0]  # Using the first answer for training
            examples.append({
                'context': context,
                'question': question,
                'answers': {
                    'text': answer['text'],
                    'answer_start': answer['answer_start']
                }
            })

# Create a Hugging Face Dataset object
squad_dataset = Dataset.from_list(examples)

c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"  # or any other pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = tokenizer(
        examples['question'],
        examples['context'],
        max_length=384,
        truncation="only_second",  # Truncate context, not question
        return_offsets_mapping=True,
        padding="max_length"
    )
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(inputs["offset_mapping"]):
        answer = examples["answers"][i]
        start_char = answer["answer_start"]
        end_char = start_char + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # If the answer is out of context range, set positions to 0
        if start_char < inputs["offset_mapping"][i][context_start][0] or end_char > inputs["offset_mapping"][i][context_end][1]:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find the token indices for the answer
            start_positions.append(
                next(idx for idx, mapping in enumerate(inputs["offset_mapping"][i]) if mapping[0] <= start_char < mapping[1])
            )
            end_positions.append(
                next(idx for idx, mapping in enumerate(inputs["offset_mapping"][i]) if mapping[0] < end_char <= mapping[1])
            )

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = squad_dataset.map(preprocess_function, batched=True, remove_columns=squad_dataset.column_names)

c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Advait\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\toke

In [3]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/16425 [00:00<?, ?it/s]c:\Users\Advait\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  3%|▎         | 500/16425 [09:11<4:53:11,  1.10s/it]

{'loss': 2.8586, 'grad_norm': 17.295013427734375, 'learning_rate': 1.9391171993911722e-05, 'epoch': 0.09}


  6%|▌         | 1000/16425 [18:25<4:43:35,  1.10s/it]

{'loss': 1.8559, 'grad_norm': 20.600038528442383, 'learning_rate': 1.8782343987823442e-05, 'epoch': 0.18}


  8%|▊         | 1261/16425 [23:17<4:39:06,  1.10s/it]